# 图像平移
原理：将原图像的坐标进行相加从而获得新图像

In [ ]:
from PIL import Image
import numpy as np
import math
import cv2

imgPath = "../assest/lean.jpg"
src_img = np.array(Image.open(imgPath).convert('L'))  # numpy.ndarray (512,512)读取图像为灰度图


def shift(shiftValue):
    matrix = np.zeros((3, 3), dtype=np.uint8)
    for i in range(0, 3):
        matrix[:, 2] = shiftValue
        matrix[i, i] = 1
    dst_img = np.zeros((1024, 1024))
    for x in range(0, 10):
        for y in range(0, 10):
            src_index = np.array([x, y, 1], ndmin=2).T
            dst_index = np.matmul(matrix, src_index)
            print(dst_index)
            if dst_index[0, 0] in range(0, 1024) and dst_index[1, 0] in range(0, 1024):
                dst_img[int(dst_index[0, 0]), int(dst_index[1, 0])] = src_img[x, y]
    result = Image.fromarray(dst_img.astype('uint8'))
    result.save("shiftResult.jpg")
    print("图像移动:", shiftValue)

# 旋转

## 为什么采用后向映射的方法放置图像？
原因：之所以有后向映射是因为在前向映射中获取的旋转后坐标是浮点数，但是像素只能是整数，所以就产生了像素缺失。
## 如何确定新图像的像素值？
采用双线性内插。

<img src=../assest/双线性插值.jpg>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.data
import skimage.io
import cv2

image_path = r"../assest/lean.jpg"
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR, cv2.CV_8UC3)


def rotate(angle):
    """
    angle:旋转角度，弧度制
    """
    Rotate_angle = angle * np.pi / 180

    # 设置新的图像大小
    h, w = img.shape[0], img.shape[1]
    newW = int(w * abs(np.cos(angle)) + h * abs(np.sin(angle))) + 1
    newH = int(w * abs(np.sin(angle)) + h * abs(np.cos(angle))) + 1

    # 目标图像
    dstImg = np.zeros((newH, newW, 3), dtype=int)

    # 设置旋转矩阵 src -> des
    trans1 = np.array([[1, 0, 0], [0, -1, 0], [-0.5 * w, 0.5 * h, 1]])
    trans1 = trans1.dot(np.array([[np.cos(angle), -np.sin(angle), 0], [np.sin(angle), np.cos(angle), 0], [0, 0, 1]]))
    trans1 = trans1.dot(np.array([[1, 0, 0], [0, -1, 0], [0.5 * newW, 0.5 * newH, 1]]))
    # des -> src
    trans2 = np.array([[1, 0, 0], [0, -1, 0], [-0.5 * newW, 0.5 * newH, 1]])
    trans2 = trans2.dot(np.array([[np.cos(angle), np.sin(angle), 0], [-np.sin(angle), np.cos(angle), 0], [0, 0, 1]]))
    trans2 = trans2.dot(np.array([[1, 0, 0], [0, -1, 0], [0.5 * w, 0.5 * h, 1]]))

    for x in range(newW):
        for y in range(newH):
            srcPos = np.array([x, y, 1]).dot(trans2)
            if srcPos[0] >= 0 and srcPos[0] < w and srcPos[1] >= 0 and srcPos[1] < h:
                # 最邻近内插
                dstImg[y][x] = img[int(srcPos[1])][int(srcPos[0])]

    # 绘图
    sub = plt.subplot(1, 2, 1)
    sub.set_title("Src Img")
    plt.imshow(img)
    sub = plt.subplot(1, 2, 2)
    sub.set_title("Des->Src & Nearest")
    plt.imshow(dstImg)
    plt.show()


rotate(45)


# 镜像

In [ ]:
from PIL import Image
import numpy as np
import math
import cv2
import copy

imgPath = "../assest/lean.jpg"
src_img = np.array(Image.open(imgPath).convert('L'))  # numpy.ndarray (512,512)读取图像为灰度图
dstImg = copy.deepcopy(src_img)
print(dstImg.shape)
imgH = src_img.shape[0]
imgW = src_img.shape[1]
for i in range(imgH):
    for j in range(imgW):
        print()
        dstImg[i][imgW - 1 - j] = src_img[i][j]
cv2.imwrite("result.jpg", dstImg)
